In [1]:
!pip install cirq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.6/594.6 KB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.4/577.4 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 KB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 KB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 KB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
import cirq
import math
from cirq.contrib.svg import SVGCircuit
import sympy
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
## DEFINITION
N = 4
D=2
A = 0.03
##sigma=np.zeros((N,N))
sigma = np.array([[9.980e-05, 4.250e-05, 3.720e-05, 4.030e-05],
                  [4.250e-05, 1.005e-04, 4.110e-05, 1.520e-05],
                  [3.720e-05, 4.110e-05, 1.813e-04, 1.790e-05],
                  [4.030e-05, 1.520e-05, 1.790e-05, 2.531e-04]])
mu = np.array([4.01e-04,  6.10e-05,  9.16e-04, -6.19e-04])
y = np.zeros(N)
qubits = [cirq.LineQubit(i) for i in range(2*N)]
L=len(qubits)
index_grid_pair = [(2*i,2*i+1) for i in range(0,N) ]

lam=0.9

In [6]:
## INITIALIZATION AND MEASURE
def circuit_hard():
    initial_hard = cirq.Circuit()
    # Initialising the first D = N/2 qubits in |10> state (s+ being the first qubit and s- being the 2nd)
    for i in range(int(D)):
            initial_hard.append(cirq.X(qubits[index_grid_pair[i][0]]))
    # Initialising the remaining N-D qubits in the |phi_00> state
    for i in range(int(D), N):
                    initial_hard.append(cirq.H(qubits[index_grid_pair[i][0]]))
                    initial_hard.append(cirq.CNOT(qubits[index_grid_pair[i][0]], qubits[index_grid_pair[i][1]]))
    return initial_hard

def circuit_hadamard():
    hadamard=cirq.Circuit()
    for i in range(L):
        hadamard.append(cirq.H(qubits[i]))
    return hadamard

def circuit_measurements():
    measurements=cirq.Circuit()
    for i in range(L):
        measurements.append(cirq.measure(qubits[i]))
    return measurements

## MIXER OPERATOR
def circuit_mixer(beta):
    mixer=cirq.Circuit()
    for i in range(L):
        mixer.append(cirq.rx(2*beta).on(qubits[i]))
    return mixer

def circuit_mixer_hard(beta):
    mixer_hard = cirq.Circuit()
    # Even parity mixer bands Ref: Fig-1 of paper
    for i in range(0,2*N-2,2):
        mixer_hard.append(cirq.ISwapPowGate(exponent=(4.0*(-beta)/np.pi)).on(qubits[i],qubits[i+2])) 
    # Odd parity mixer bands Ref: Fig-1 of paper
    for i in range(1,2*N-2,2):
        mixer_hard.append(cirq.ISwapPowGate(exponent=(4.0*(-beta)/np.pi)).on(qubits[i],qubits[i+2]))
    # Long position parity mixer Ref: Fig-1 of paper
    mixer_hard.append(cirq.ISwapPowGate(exponent=(4.0*(-beta)/np.pi)).on(qubits[2*N-1],qubits[1]))
    # Short position parity mixer Ref: Fig-1 of paper
    mixer_hard.append(cirq.ISwapPowGate(exponent=(4.0*(-beta)/np.pi)).on(qubits[2*N-2],qubits[0]))
    return mixer_hard

## RETURNS OPERATOR
def circuit_returns(gamma):
    returns=cirq.Circuit()
    for i in range(L):
        returns.append(cirq.rz((-1)**int(i)*(1-lam)*gamma*mu[int(i/2)]).on(qubits[i]))
    return returns

## RISK OPERATOR
def circuit_risk(gamma):
    combinations=[]
    for i in range(L):
        for j in range(L):
            if i!=j:
                combinations.append((i,j))
    numcombinations=len(combinations)
    risk=cirq.Circuit()
    for i in range(numcombinations):
        q1=combinations[i][0]
        q2=combinations[i][1]
        ii=math.floor(q1/2)
        jj=math.floor(q2/2)
        sign=-(-1)**((q1 % 2)+(q2 %2))
        risk.append(cirq.ZZPowGate(exponent= 1/2*sign*gamma*lam*sigma[ii][jj]).on(qubits[q1],qubits[q2]))
    return risk

##  CONSTRAINT OPERATOR
def circuit_constraint(gamma):
    combinations=[]
    for i in range(L):
        for j in range(L):
            if i!=j:
                combinations.append((i,j))
    numcombinations=len(combinations)
    constraint=cirq.Circuit()
    for i in range(len(qubits)):
        sign=(-1)**i
        constraint.append(cirq.rz( sign *2 *gamma*A*D).on(qubits[i]))
    for i in range(numcombinations):
        q1=combinations[i][0]
        q2=combinations[i][1]
        sign=-(-1)**((q1 % 2)+(q2 %2))
        constraint.append(cirq.ZZPowGate(exponent= 1/2*sign*A*gamma).on(qubits[q1],qubits[q2]))
    return constraint

## CIRCUIT BUILDER
def circuit_builder(beta,gamma):
    hadamard=circuit_hadamard()
    returns=circuit_returns(gamma)
    risk=circuit_risk(gamma)
    constraint=circuit_constraint(gamma)
    mixer=circuit_mixer(beta)
    measurements=circuit_measurements()
    return cirq.Circuit(hadamard, returns, risk, constraint, mixer, measurements)

# def circuit_builder_p(beta,gamma): #beta and gamma are arrays of length
#     if len(beta)==len(gamma):
#         p=len(beta)
#     else:
#         print("Error: beta and gamma must have the same length")
#         return 1;
#     circuit=cirq.Circuit()
#     circuit.append(circuit_hadamard())
#     for i in range(p):
#         circuit.append(circuit_returns(gamma[i]))
#         circuit.append(circuit_risk(gamma[i]))
#         circuit.append(circuit_constraint(gamma[i]))
#         circuit.append(circuit_mixer(beta[i]))
#     circuit.append(circuit_measurements())
#     return circuit

def circuit_builder_p_hard(beta,gamma): #Creates a circuit of depth p (beta and gamma must be arrays of length p)
    if len(beta)==len(gamma):
        p=len(beta)
    else:
        print("Error: beta and gamma must have the same length")
        return 1;
    circuit=cirq.Circuit()
    circuit.append(circuit_hard())
    for i in range(p):
        circuit.append(circuit_returns(gamma[i]))
        circuit.append(circuit_risk(gamma[i]))
        circuit.append(circuit_mixer_hard(beta[i]))
    circuit.append(circuit_measurements())
    return circuit

## COST FUNCTION
# def cost_function(z):
#     return lam*np.dot(z,np.dot(sigma,z))+A*(D-sum(z))**2-(1-lam)*np.dot(mu,z)

def cost_function_hard(z):
    return lam*np.dot(z,np.dot(sigma,z))-(1-lam)*np.dot(mu,z)

# def cost_function_angle(beta,gamma,numsim):
#     simulator = cirq.Simulator()
#     Cost_min=100
#     circuit=circuit_builder_p(beta,gamma)
#     for i in range(numsim):
#         result = simulator.run(circuit)
#         list=[]
#         z=[]
#         for value in result.measurements.values():
#             list.append(value[0,0])
#         for j in range(N):
#             z.append((list[2*j]-list[2*j+1]))
#         C=cost_function(z)
#         if C<Cost_min:
#             Cost_min=C
#             positions=z
#     return positions, Cost_min

def cost_function_angle_hard(beta,gamma,numsim):
    simulator = cirq.Simulator()
    Cost_min=100
    circuit=circuit_builder_p_hard(beta,gamma)
    for i in range(numsim):
        result = simulator.run(circuit)
        list=[]
        z=[]
        for value in result.measurements.values():
            list.append(value[0,0])
        for j in range(N):
            z.append((list[2*j]-list[2*j+1]))
        C=cost_function_hard(z)
        if C<Cost_min:
            Cost_min=C
            positions=z
    return positions, Cost_min

# def cost_function_angle_mean(beta,gamma,numsim):
#     simulator = cirq.Simulator()
#     circuit=circuit_builder_p(beta,gamma)
#     C=[]
#     for i in range(numsim):
#         result = simulator.run(circuit)
#         list=[]
#         z=[]
#         for value in result.measurements.values():
#             list.append(value[0,0])
#         for j in range(N):
#             z.append((list[2*j]-list[2*j+1]))
#         C.append(cost_function(z))
#     return np.mean(C)

def cost_function_angle_mean_hard(beta,gamma,numsim):
    simulator = cirq.Simulator()
    circuit=circuit_builder_p_hard(beta,gamma)
    C=[]
    for i in range(numsim):
        result = simulator.run(circuit)
        list=[]
        z=[]
        for value in result.measurements.values():
            list.append(value[0,0])
        for j in range(N):
            z.append((list[2*j]-list[2*j+1]))
        C.append(cost_function_hard(z))
    return np.mean(C)

## GRID SEARCH
# def grid_search(numpoints, numsim):
#     Cost_min=100
#     simulator = cirq.Simulator()
#     positions=[]
#     betas= np.linspace(0, 2 * np.pi, numpoints)
#     gammas= np.linspace(0, 2 * np.pi, numpoints)
#     for beta in betas:
#         for gamma in gammas:
#             circuit=circuit_builder(beta,gamma)
#             for i in range(numsim):
#                 result = simulator.run(circuit)
#                 list=[]
#                 z=[]
#                 for value in result.measurements.values():
#                     list.append(value[0,0])
#                 for i in range(N):
#                     z.append((list[2*i]-list[2*i+1]))
#                 C=cost_function(z)
#                 if C<Cost_min:
#                     Cost_min=C
#                     positions=z
#     return positions, Cost_min

# def grid_search_2(numpoints, numsim):
#     Cost_min=100
#     simulator = cirq.Simulator()
#     positions=[]
#     betas= np.linspace(0, 2 * np.pi, numpoints)
#     gammas= np.linspace(0, 2 * np.pi, numpoints)
#     for beta1 in betas:
#         for gamma1 in gammas:
#             for beta2 in betas:
#                 for gamma2 in gammas:
#                     circuit=circuit_builder_p([beta1,beta2],[gamma1,gamma2])
#                     for i in range(numsim):
#                         result = simulator.run(circuit)
#                         list=[]
#                         z=[]
#                         for value in result.measurements.values():
#                             list.append(value[0,0])
#                         for i in range(N):
#                             z.append((list[2*i]-list[2*i+1]))
#                         C=cost_function(z)
#                         if C<Cost_min:
#                             Cost_min=C
#                             positions=z
#     return positions, Cost_min

# def cross_entropy_mean(p,iterations,Nce_samples,f_elite):
#     width = np.identity(2*p)
#     centre = np.random.uniform(0.0,2*np.pi,2*p)
#     print("MEAN")
#     for i in range(iterations):
#         # Generate N-samples from the multivariate Gaussian
#         X = np.random.multivariate_normal(centre,width,Nce_samples)
#         E = np.zeros(len(X))
#         data = []
        
#         for k in range(len(X)):
#             E[k] = cost_function_angle_mean(X[k][0:p],X[k][p:],20)
#             data.append([*X[k],E[k]])

#         # Sort the value according to the best
#         sorted_data = np.array(sorted(data, key=lambda x: x[2*p], reverse=False))

#         # Now compute the new averages
#         N_elite = int(f_elite*Nce_samples)
#         X_elite = sorted_data[0:N_elite,0:2*p]
#         E_elite = sorted_data[0:N_elite,2*p:]
        
#         print('k={}, mean_E={:.6f}, std_E={:.6f}, beta={}, gamma={}'.format(i,E_elite.mean(),E_elite.std(),centre[0:p],centre[p:]))

#         # Compute the new values of mu and sigma
#         centre = np.mean(X_elite,axis=0)
#         width = np.cov(X_elite.T)

def cross_entropy_mean_hard(p,iterations,Nce_samples,f_elite):
    width = np.identity(2*p)
    centre = np.random.uniform(0.0,2*np.pi,2*p)
    print("MEAN")
    for i in range(iterations):
        # Generate N-samples from the multivariate Gaussian
        X = np.random.multivariate_normal(centre,width,Nce_samples)
        E = np.zeros(len(X))
        data = []
        
        for k in range(len(X)):
            E[k] = cost_function_angle_mean_hard(X[k][0:p],X[k][p:],20)
            data.append([*X[k],E[k]])

        # Sort the value according to the best
        sorted_data = np.array(sorted(data, key=lambda x: x[2*p], reverse=False))

        # Now compute the new averages
        N_elite = int(f_elite*Nce_samples)
        X_elite = sorted_data[0:N_elite,0:2*p]
        E_elite = sorted_data[0:N_elite,2*p:]
        
        print('k={}, mean_E={:.6f}, std_E={:.6f}, beta={}, gamma={}'.format(i,E_elite.mean(),E_elite.std(),centre[0:p],centre[p:]))

        # Compute the new values of mu and sigma
        centre = np.mean(X_elite,axis=0)
        width = np.cov(X_elite.T)


    # get the other results
    betas = centre[0:p]
    gammas = centre[p:]
    return betas, gammas
    
# def cross_entropy_min(p,iterations,Nce_samples,f_elite):
#     width = np.identity(2*p)
#     centre = np.random.uniform(0.0,2*np.pi,2*p)
#     print("MIN")
#     for i in range(iterations):
#         # Generate N-samples from the multivariate Gaussian
#         X = np.random.multivariate_normal(centre,width,Nce_samples)
#         E = np.zeros(len(X))
#         data = []
        
#         for k in range(len(X)):
#             E[k] = cost_function_angle(X[k][0:p],X[k][p:],10)[1]
#             data.append([*X[k],E[k]])

#         # Sort the value according to the best
#         sorted_data = np.array(sorted(data, key=lambda x: x[2*p], reverse=False))

#         # Now compute the new averages
#         N_elite = int(f_elite*Nce_samples)
#         X_elite = sorted_data[0:N_elite,0:2*p]
#         E_elite = sorted_data[0:N_elite,2*p:]
        
#         print('k={}, mean_E={:.6f}, std_E={:.6f}, beta={}, gamma={}'.format(i,E_elite.mean(),E_elite.std(),centre[0:p],centre[p:]))

#         # Compute the new values of mu and sigma
#         centre = np.mean(X_elite,axis=0)
#         width = np.cov(X_elite.T)

def cross_entropy_min_hard(p,iterations,Nce_samples,f_elite):
    width = np.identity(2*p)
    centre = np.random.uniform(0.0,2*np.pi,2*p)
    print("MIN")
    for i in range(iterations):
        # Generate N-samples from the multivariate Gaussian
        X = np.random.multivariate_normal(centre,width,Nce_samples)
        E = np.zeros(len(X))
        data = []
        
        for k in range(len(X)):
            E[k] = cost_function_angle_hard(X[k][0:p],X[k][p:],10)[1]
            data.append([*X[k],E[k]])

        # Sort the value according to the best
        sorted_data = np.array(sorted(data, key=lambda x: x[2*p], reverse=False))

        # Now compute the new averages
        N_elite = int(f_elite*Nce_samples)
        X_elite = sorted_data[0:N_elite,0:2*p]
        E_elite = sorted_data[0:N_elite,2*p:]
        
        print('k={}, mean_E={:.6f}, std_E={:.6f}, beta={}, gamma={}'.format(i,E_elite.mean(),E_elite.std(),centre[0:p],centre[p:]))

        # Compute the new values of mu and sigma
        centre = np.mean(X_elite,axis=0)
        width = np.cov(X_elite.T)

    # get the other results
    betas = centre[0:p]
    gammas = centre[p:]

    return betas,gammas

# def quality_parameters(beta,gamma,E_treshold,R):
#     simulator = cirq.Simulator()
#     p=0
#     q=0
#     circuit=circuit_builder_p(beta,gamma)
#     for i in range(R):
#         result = simulator.run(circuit)
#         list=[]
#         z=[]
#         for value in result.measurements.values():
#             list.append(value[0,0])
#         for i in range(N):
#             z.append((list[2*i]-list[2*i+1]))
#         C=cost_function(z)
#         if(C<E_treshold):
#             p+=1
#         if(z==[1,0,1,0]):
#             q+=1
#     return p/R,q/R,cost_function_angle_mean(beta,gamma,R)

def quality_parameters_hard(beta,gamma,E_treshold,R):
    simulator = cirq.Simulator()
    p=0
    q=0
    circuit=circuit_builder_p_hard(beta,gamma)
    for i in range(R):
        result = simulator.run(circuit)
        list=[]
        z=[]
        for value in result.measurements.values():
            list.append(value[0,0])
        for i in range(N):
            z.append((list[2*i]-list[2*i+1]))
        C=cost_function_hard(z)
        if(C<E_treshold):
            p+=1
        if(z==[1,0,1,0]):
            q+=1
    return p/R,q/R,cost_function_angle_mean_hard(beta,gamma,R)

In [5]:

p=1   #Circuit depth # 1,2,3
iterations=10; # Number of iterations for cross entropy 1,5,10,20
Nce_samples=10; # Number of samples 10,15,20
f_elite=0.2;   # fraction of best samples 0.2,0.4, 1

Num=10# Number of times we run the loop 
E_treshold=0.005; # Maximum energy which we tolerate
R=1000; # Number of times we average the quality parameters
beta_min=np.zeros((Num,p)) #Circuit parameters beta and gamma for min strategy
gamma_min=np.zeros((Num,p))
beta_mean=np.zeros((Num,p)) #Circuit parameters beta and gamma for mean strategy
gamma_mean=np.zeros((Num,p))
Q_min=np.zeros((Num,3))    # Quality parameters for min strategy
Q_mean=np.zeros((Num,3))   # Quality parameters for mean strategy

for ii in range(Num):
    beta_min[ii],gamma_min[ii]=cross_entropy_min(p,iterations,Nce_samples,f_elite)
    beta_mean[ii],gamma_mean[ii]=cross_entropy_mean(p,iterations,Nce_samples,f_elite)
    Q_min[ii]=quality_parameters(beta_min[ii],gamma_min[ii],E_treshold,R)
    Q_mean[ii]=quality_parameters(beta_mean[ii],gamma_mean[ii],E_treshold,R)
    
beta_min,gamma_min,Q_min, Q_mean

MIN
k=0, mean_E=0.000211, std_E=0.000000, beta=[0.8184669], gamma=[0.11332455]
k=1, mean_E=0.000199, std_E=0.000011, beta=[0.64173229], gamma=[-0.03498012]
k=2, mean_E=0.000291, std_E=0.000103, beta=[0.74216265], gamma=[-0.03765256]
k=3, mean_E=0.000209, std_E=0.000021, beta=[0.83542222], gamma=[-0.04013418]
k=4, mean_E=0.000188, std_E=0.000000, beta=[0.83527659], gamma=[-0.0401303]
k=5, mean_E=0.000211, std_E=0.000000, beta=[0.83659428], gamma=[-0.04016537]
k=6, mean_E=0.000188, std_E=0.000000, beta=[0.83868647], gamma=[-0.04022104]
k=7, mean_E=0.000199, std_E=0.000011, beta=[0.83556756], gamma=[-0.04013805]
k=8, mean_E=0.000211, std_E=0.000000, beta=[0.83556403], gamma=[-0.04013795]
k=9, mean_E=0.000220, std_E=0.000010, beta=[0.83556725], gamma=[-0.04013804]


TypeError: ignored

In [ ]:
beta_min_p1_iter10_Nce10_f02_soft=beta_min
gamma_min_p1_iter10_Nce10_f02_soft=gamma_min
beta_mean_p1_iter10_Nce10_f02_soft=beta_mean
gamma_mean_p1_iter10_Nce10_f02_soft=gamma_mean
Q_min_p1_iter10_Nce10_f02_soft=Q_min
Q_mean_p1_iter10_Nce10_f02_soft=Q_mean
np.savetxt('beta_min_p1_iter10_Nce10_f02_soft.csv', beta_min_p1_iter10_Nce10_f02_soft, delimiter=',')
np.savetxt('beta_mean_p1_iter10_Nce10_f02_soft.csv', beta_mean_p1_iter10_Nce10_f02_soft, delimiter=',')
np.savetxt('gamma_min_p1_iter10_Nce10_f02_soft.csv', gamma_min_p1_iter10_Nce10_f02_soft, delimiter=',')
np.savetxt('gamma_mean_p1_iter10_Nce10_f02_soft.csv', gamma_mean_p1_iter10_Nce10_f02_soft, delimiter=',')
np.savetxt('Q_min_p1_iter10_Nce10_f02_soft.csv', Q_min_p1_iter10_Nce10_f02_soft, delimiter=',')
np.savetxt('Q_mean_p1_iter10_Nce10_f02_soft.csv', Q_mean_p1_iter10_Nce10_f02_soft, delimiter=',')

In [ ]:
# How to load data
data = np.loadtxt('beta_min_p1_iter1_Nce5_f1_soft.csv', delimiter=',')

In [ ]:
## HARD

p=2   #Circuit depth # 1,2,3
iterations=1; # Number of iterations for cross entropy 1,5,10,20
Nce_samples=10; # Number of samples 10,15,20
f_elite=0.2;   # fraction of best samples 0.2,0.4, 1

Num=10# Number of times we run the loop 
E_treshold=0.005; # Maximum energy which we tolerate
R=1000; # Number of times we average the quality parameters
beta_min=np.zeros((Num,p)) #Circuit parameters beta and gamma for min strategy
gamma_min=np.zeros((Num,p))
beta_mean=np.zeros((Num,p)) #Circuit parameters beta and gamma for mean strategy
gamma_mean=np.zeros((Num,p))
Q_min=np.zeros((Num,3))    # Quality parameters for min strategy
Q_mean=np.zeros((Num,3))   # Quality parameters for mean strategy

for ii in range(Num):
    beta_min[ii],gamma_min[ii]=cross_entropy_min_hard(p,iterations,Nce_samples,f_elite)
    beta_mean[ii],gamma_mean[ii]=cross_entropy_mean_hard(p,iterations,Nce_samples,f_elite)
    Q_min[ii]=quality_parameters_hard(beta_min[ii],gamma_min[ii],E_treshold,R)
    Q_mean[ii]=quality_parameters_hard(beta_mean[ii],gamma_mean[ii],E_treshold,R)
    
beta_min,gamma_min,Q_min, Q_mean

MIN
k=0, mean_E=0.000188, std_E=0.000000, beta=[5.87302942 2.14024978], gamma=[0.97451891 1.08577706]
MEAN
k=0, mean_E=0.000278, std_E=0.000018, beta=[5.29478649 3.23714212], gamma=[4.79925286 0.40014055]
MIN
k=0, mean_E=0.000188, std_E=0.000000, beta=[2.22404208 3.78868374], gamma=[0.75879609 1.2748296 ]
MEAN
k=0, mean_E=0.000199, std_E=0.000011, beta=[2.26070469 2.35342856], gamma=[1.21553224 5.60255697]
MIN
k=0, mean_E=0.000188, std_E=0.000000, beta=[2.85179001 2.40165007], gamma=[3.34983572 0.59215662]
MEAN
k=0, mean_E=0.000228, std_E=0.000004, beta=[2.10388316 0.80300107], gamma=[4.72719568 1.63568471]
MIN
k=0, mean_E=0.000188, std_E=0.000000, beta=[0.57231049 0.93505399], gamma=[3.87387737 5.58015667]
MEAN
k=0, mean_E=0.000223, std_E=0.000001, beta=[0.76266008 1.82869929], gamma=[0.96531605 1.38851014]
MIN
k=0, mean_E=0.000188, std_E=0.000000, beta=[5.90215785 2.85303758], gamma=[2.02418157 5.0865251 ]
MEAN
k=0, mean_E=0.000224, std_E=0.000025, beta=[3.00275616 0.91211494], gamma

In [ ]:
## HARD
beta_min_p2_iter1_Nce10_f02_hard=beta_min
gamma_min_p2_iter1_Nce10_f02_hard=gamma_min
beta_mean_p2_iter1_Nce10_f02_hard=beta_mean
gamma_mean_p2_iter1_Nce10_f02_hard=gamma_mean
Q_min_p2_iter1_Nce10_f02_hard=Q_min
Q_mean_p2_iter1_Nce10_f02_hard=Q_mean
np.savetxt('beta_min_p2_iter1_Nce10_f02_hard.csv', beta_min_p2_iter1_Nce10_f02_hard, delimiter=',')
np.savetxt('beta_mean_p2_iter1_Nce10_f02_hard.csv', beta_mean_p2_iter1_Nce10_f02_hard, delimiter=',')
np.savetxt('gamma_min_p2_iter1_Nce10_f02_hard.csv', gamma_min_p2_iter1_Nce10_f02_hard, delimiter=',')
np.savetxt('gamma_mean_p2_iter1_Nce10_f02_hard.csv', gamma_mean_p2_iter1_Nce10_f02_hard, delimiter=',')
np.savetxt('Q_min_p2_iter1_Nce10_f02_hard.csv', Q_min_p2_iter1_Nce10_f02_hard, delimiter=',')
np.savetxt('Q_mean_p2_iter1_Nce10_f02_hard.csv', Q_mean_p2_iter1_Nce10_f02_hard, delimiter=',')